# Leather Detection for Mask RCNN


In [2]:
import os
import sys
import cv2
import time
import imutils
import numpy as np
import mrcnn.model as modellib
from mrcnn import utils, visualize
from imutils.video import WebcamVideoStream
import random
from threading import Thread
from queue import Queue
import datetime

# Root directory of the project
from mrcnn.config import Config
# 根目錄
ROOT_DIR = os.path.abspath("./")
# 系統路徑設定
sys.path.append(os.path.join(ROOT_DIR, "samples/leather/"))  # To find local version

out_filename = '' # video output file name
# Directory to save logs and trained model
# 模型儲存目錄
MODEL_DIR = os.path.join(ROOT_DIR, "logs") 
print(MODEL_DIR)
#video_input = "D:/Project/yolov4/Mango1/data/data/2019_tw_run.mp4" #"video source. If video_input=0, uses webcam 0 stream"

# 鏡頭選擇
video_input = 0 #usb cam device 0

Instructions for updating:
non-resource variables are not supported in the long term
ROOT:  D:\Project\Mask_RCNN
D:\Project\Mask_RCNN\logs


## 設定權重檔路徑(Weights file)
## 將Config導入，預設值來自於mrcnn/config.py 以及本地目錄中的config.ini，如果Mask RCNN已經安裝到虛擬環境，則需要到虛擬環境下的`<envs>\Lib\site-packages\mask_rcnn-2.1-py3.7.egg`
### 打包[egg](https://blog.csdn.net/caiguoxiong0101/article/details/50285279) ，完成後將Mask RCNN目錄 `dist/`下面的 *.egg 覆蓋到安裝路徑上。

In [3]:
# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(MODEL_DIR, 'shapes20201218T1013',"mask_rcnn_shapes_0150.h5")
# Download COCO trained weights from Releases if needed
#if not os.path.exists(COCO_MODEL_PATH):
#    utils.download_trained_weights(COCO_MODEL_PATH)
if os.path.exists(COCO_MODEL_PATH):
    print('COCO_MODEL_PATH: PASS', COCO_MODEL_PATH)

class ShapesConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "shapes"

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 9  # background + 1 shapes #如果你有超過一個label, 記得改1 + 2或者以上

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 384
    IMAGE_MAX_DIM = 448

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8 * 6, 16 * 6, 32 * 6, 64 * 6, 128 * 6)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 100

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 100

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 50


config = ShapesConfig()
onfig.display()


COCO_MODEL_PATH: PASS D:\Project\Mask_RCNN\logs\shapes20201218T1013\mask_rcnn_shapes_0150.h5

Configurations:
AUGMENT                        False
AUGMENTATION                   False
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  448
IMAGE_META_SIZE                22
IMAGE_MIN_DIM                  384
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [448 448   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_cla

# 模型Mask RCNN導入
## 判斷畫面上的瑕疵並將結果輸出內容顯示在畫面上

In [5]:
SCREEN_NAME = 'Mask RCNN LIVE' # CV2 Windw name
# CV2全螢幕顯示
#cv2.namedWindow(SCREEN_NAME, cv2.WINDOW_NORMAL)
#cv2.setWindowProperty(SCREEN_NAME, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

#width = config.IMAGE_MAX_DIM
#height = config.IMAGE_MIN_DIM

def CapVideo(frame):

    #if SHOW_FPS_WO_COUNTER:
    start_time = time.time()  # start time of the loop
    cv2.imshow('Before', frame)
    
    if PROCESS_IMG:
        results = model.detect([frame])
        r = results[0]

        # Run detection
        masked_image = visualize.display_instances_10fps(frame, r['rois'], r['masks'],
                                                         r['class_ids'], class_names, r['scores'], colors=colors,
                                                         real_time=True)
    

    if PROCESS_IMG:
        s = masked_image
    else:
        s = frame
    # print("Image shape: {1}x{0}".format(s.shape[0], s.shape[1]))

    width = s.shape[1]
    height = s.shape[0]
    top_left_corner = (width - 120, height - 20)
    bott_right_corner = (width, height)
    top_left_corner_cvtext = (width - 80, height - 5)

    if SHOW_FPS:
        fps_counter += 1
        if (time.time() - start_time) > 5:  # every 5 second
            fps_caption = "FPS: {:.0f}".format(fps_counter / (time.time() - start_time))
            # print(fps_caption)
            fps_counter = 0
            start_time = time.time()
        ret, baseline = cv2.getTextSize(fps_caption, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        cv2.rectangle(s, (width - ret[0], height - ret[1] - baseline), bott_right_corner, gentle_grey, -1)
        cv2.putText(s, fps_caption, (width - ret[0], height - baseline), cv2.FONT_HERSHEY_SIMPLEX, 0.5, white,
                    lineType=cv2.LINE_AA)

    if SHOW_FPS_WO_COUNTER:
        # Display the resulting frame
        fps_caption = "FPS: {:.0f}".format(1.0 / (time.time() - start_time))
        # print("FPS: ", 1.0 / (time.time() - start_time))

        # Put the rectangle and text on the bottom left corner
        ret, baseline = cv2.getTextSize(fps_caption, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        cv2.rectangle(s, (width - ret[0], height - ret[1] - baseline), bott_right_corner, gentle_grey, -1)
        cv2.putText(s, fps_caption, (width - ret[0], height - baseline),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, white, 1, lineType=cv2.LINE_AA)

    s = cv2.resize(s, (640, 480))

    return SCREEN_NAME, s




# 抓取USB輸入影像

In [6]:
class VideoGet:
    """
    Class that continuously gets frames from a VideoCapture object
    with a dedicated thread.
    """

    def __init__(self, src=0):
        self.stream = cv2.VideoCapture(src + cv2.CAP_DSHOW)
        #self.stream = cv2.VideoCapture(src)
        (self.grabbed, self.frame) = self.stream.read()
        self.stopped = False
        
    def start(self):    
        Thread(target=self.get, args=()).start()
        return self

    def get(self):
        while not self.stopped:
            if not self.grabbed:
                self.stop()
            else:
                (self.grabbed, self.frame) = self.stream.read()

    def stop(self):
        self.stopped = True

## 搭配VideoGet()，將CV2取到的畫面放到Mutil-process處理
## 再將影像送到模型中進行判斷，可以加快整體運作速度。
## `q`則結束

In [7]:
def threadVideoGet(source=0):
    """
    Dedicated thread for grabbing video frames with VideoGet object.
    Main thread shows video frames.
    """

    video_getter = VideoGet(source).start()
    #cps = CountsPerSec().start()

    while True:
        if (cv2.waitKey(1) == ord("q")) or video_getter.stopped:
            video_getter.stop()
            break

        frame = video_getter.frame
        name, frame = CapVideo(frame)

        cv2.imshow(name, frame)
        #cps.increment()

# 程式主流程
## 1. 載入模型權重
## 2. 定義輸出類別名稱，同訓練時名稱
## 3. 將類別顏色使用亂數選擇
## 4. 設定參數
## 5. 執行threadVideoGet()程式直到`q`被按下
## 6. 停止輸入影像
## 7. 停止釋放串流與關閉視窗。

In [ ]:
# Create model object in inference mode.
global model
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG','scratch', 'scratch2','Wrinkles', 'Wrinkles2','vein', 'vein2',
               'GrowthPattern','dot','scratch3']

colors = visualize.random_colors(len(class_names))

gentle_grey = (45, 65, 79)
white = (255, 255, 255)

OPTIMIZE_CAM = False
SHOW_FPS = False
SHOW_FPS_WO_COUNTER = True  # faster
PROCESS_IMG = True

#if OPTIMIZE_CAM:
#    vs = WebcamVideoStream(src=0).start()

if SHOW_FPS:
    fps_caption = "FPS: 0"
    global fps_counter
    fps_counter = 0
    global start_time
    start_time = time.time()

threadVideoGet(video_input)
video_getter = VideoGet(video_input).start()
video_getter.stop()
video_getter.stream.release()
cv2.destroyAllWindows()

Re-starting from epoch 150


## 確認GPU是否有啟動，某些Jupyter環境啟動後，GPU可能是沒有動作的，但因為Tensorflow2.0之後CPU跟GPU是在同一個版本中，如果沒有特別指定，則是會自動切換。
## Mask RCNN若如GPU支援，則判斷速度FPS可能再>1 以下。

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15058091407120069240
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 15836223316918203098
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5060693856
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14154384174669849841
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 9161432300683373942
physical_device_desc: "device: XLA_GPU device"
]


In [ ]:
import tensorflow as tf
import os
#os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
a = tf.constant(1.)
b = tf.constant(2.)
print(a+b)
print(tf.__version__)
print('GPU:', tf.config.list_physical_devices('GPU'))
print(tf.test.is_gpu_available())

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
cpus = tf.config.experimental.list_physical_devices(device_type='CPU')
print(gpus, cpus)